<a href="https://colab.research.google.com/github/bdu-birhanu/HHD-Ethiopic/blob/main/train_HPopt_Attn_CTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tranining an attention agumented CTC model


### Importing Libraries:

Begin by importing the necessary libraries and frameworks include TensorFlow, Keras, and  any other libraries as provided below.

In [9]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Bidirectional, MaxPool2D
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Permute, Flatten, Masking, \
    GaussianNoise, Reshape, Lambda, TimeDistributed, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import RepeatVector, Multiply, Dot, Bidirectional,LSTM, Input, Dense, Activation,Lambda,Concatenate
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Download the training data

Download the training data from Hugging Face. The dataset is in a format of .npy and it is compatible with this platform.


In [4]:
import requests
import zipfile

# Download the zip file from Hugging Face (when coping ht elink just replace "blob" by "resolve" so as to reference the file contents.)
url="https://huggingface.co/datasets/OCR-Ethiopic/HHD-Ethiopic/resolve/main/train/train_numpy.zip"
response = requests.get(url)

# Save the zip file
zip_path = "/content/train_numpy.zip"
with open(zip_path, "wb") as file:
    file.write(response.content)

# Extract the zip file
extract_path = "/content/train"
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

num_class=307
maxlen=46
im_row=48
im_col=368

In [5]:
def load_dataset():
    '''
    '''
    x_train_real= np.load('/content/train/train_numpy/x_train.npy',allow_pickle=True)
    y_train_padded= np.load('/content/train/train_numpy/y_train.npy',allow_pickle=True)
    

    return x_train_real, y_train_padded 



In [6]:
def resize():
    data_loaded=load_dataset()
    x_train_real=data_loaded[0]
    y_train_real = data_loaded[1]

    resized_x_rand = np.zeros((len(x_train_real), im_row, im_col), dtype=np.uint8)

    # loop over the input images
    for i, image in enumerate(x_train_real):
        # resize the image to 48 by 368 using padding
        current_height, current_width = image.shape[:2]
        aspect_ratio_current = current_width / current_height
        aspect_ratio_target = im_col / im_row
        if aspect_ratio_current != aspect_ratio_target:
            if aspect_ratio_current > aspect_ratio_target:
                new_height = int(current_width / aspect_ratio_target)
                top_padding = (new_height - current_height) // 2
                bottom_padding = new_height - current_height - top_padding
                padded_image = cv2.copyMakeBorder(image, top_padding, bottom_padding, 0, 0, cv2.BORDER_CONSTANT, value=255)
            else:
                new_width = int(current_height * aspect_ratio_target)
                left_padding = (new_width - current_width) // 2
                right_padding = new_width - current_width - left_padding
                padded_image = cv2.copyMakeBorder(image, 0, 0, 0, right_padding+left_padding, cv2.BORDER_CONSTANT, value=255)
        else:
            padded_image = image
        resized_image = cv2.resize(padded_image, (im_col, im_row))

        resized_x_rand[i] = resized_image

    return resized_x_rand, y_train_real

In [7]:
def preprocess_train_val_test_data():
    ''' 
    input: a 2D shape text-line image (h,w)
    output:  returns 3D shape image format (h,w,1)

    Plus this function randomly splits the training and validation set
    This function also computes list of length for both training and validation images and GT
      '''
    #the function resize is excuted here
    x_tr=resize()
    x_train_real_resize=x_tr[0]
    y_train_real = x_tr[1]
  

    x_train, x_val, y_train, y_val = train_test_split(x_train_real_resize, y_train_real, test_size=0.1)

    # reshape the size of the image from 3D to 4D so as to make the input size is similar with it.
    x_train_r = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)  # [samplesize,32,128,1]
    x_val_r = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], 1)
    y_train = y_train
    y_val = y_val

    nb_train = len(x_train_r)
    nb_val = len(x_val_r)


    x_train_len = np.array([len(x_train_r[i]) + 43 for i in range(nb_train)])
    # the + 43 here is just to make  balanceed size ((2*46-1=91) then 48+43=91)of the image equal to the input of LSTMlayer
    x_val_len = np.array([len(x_val_r[i]) + 43 for i in range(nb_val)])  
    
    y_train_len = np.array([len(y_train[i]) for i in range(nb_train)])
    y_val_len = np.array([len(y_val[i]) for i in range(nb_val)])


    return x_train_r, y_train, x_train_len, y_train_len, x_val, y_val, x_val_len, y_val_len
'''
all set of text images and GT
'''
train=preprocess_train_val_test_data()
x_train=train[0]
y_train=train[1]
x_train_length=train[2]
y_train_length=train[3]

x_val=train[4]
y_val=train[5]
x_val_length=train[6]
y_val_length=train[7]


print("data_loading is compeletd")
print("===============================")
print(str(len(x_train))+ " train image and "+ str(len(y_train))+" labels")
print(str(len(x_val))+ "valid image and "+ str(len(y_val))+ "labels")

data_loading is compeletd
51636 train image and 51636 labels
5738valid image and 5738labels


### Model definition of the attention module


In [10]:
dot_pro = Dot(axes = 1)
concatenat = Concatenate(axis=-1)

In [12]:
def attention(blstm_2):
    score = Dense(91, activation='tanh', name='attention_score_vec')(blstm_2)

    attention_weights = Activation('softmax', name='attention_weight')(score)

   
    context_vector = dot_pro([attention_weights,blstm_2])
    

    return  context_vector


### A function for CTC

In [13]:
def ctc_lambda_func(args):
    '''

    '''
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Model definition of the CNN-LSTM-CTC

In [15]:
def hist_ocr_model(config: dict, n_components: int = 5, verbose: bool = 0, num_class=num_class, img_row=im_row, img_col=im_col):
    tf.keras.utils.set_random_seed(2)

    default_config = {
        "rnn_size": 128,
        "feature_map_1": 64,
        "feature_map_2": 128,
        "activation": "relu",
        "batch_size": 32,
        "epoch": 10,
        "kernel": 3,
        "dropout": 0.25,

    }
    default_config.update(config)
    '''
    if you use the full datset you could increase the batch_size and epo
    '''
    
    inputs_data = Input(shape=(img_row, img_col, 1))
    conv_1 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
                    activation=default_config["activation"], padding='same')(inputs_data)
    
    pool_1 = MaxPool2D(pool_size=(2, 1), strides=2)(conv_1)
    conv_2 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
                    activation=default_config["activation"], padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2))(conv_2)  # we remove the strides here
    conv_3 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
                    activation=default_config["activation"], padding='same')(pool_2)
    conv_4 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
                    activation=default_config["activation"], padding='same')(conv_3)
    
    pool_3 = MaxPool2D(pool_size=(2, 1))(conv_4)
    
    batch_norm_5 = BatchNormalization()(pool_3)
    conv_5 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
                    activation=default_config["activation"], padding='same')(batch_norm_5)

    conv_6 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
                    activation=default_config["activation"], padding='same')(conv_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    # conv_7 = Conv2D(default_config["feature_map_1"], (default_config["kernel"], default_config["kernel"]),
    #                 activation=default_config["activation"], padding='same')(batch_norm_6)
    conv_7 = Conv2D(128, (2, 2), activation='relu')(batch_norm_6)
    r = Reshape((int(conv_7.shape[2]), int(conv_7.shape[1]) * int(conv_7.shape[3])))(conv_7)

    blstm_1 = Bidirectional(LSTM(default_config["rnn_size"], return_sequences=True, dropout=default_config["dropout"]))(r)
    blstm_2 = Bidirectional(LSTM(default_config["rnn_size"], return_sequences=True, dropout=default_config["dropout"]))(blstm_1)


    context = attention(blstm_2)
    outputs = Dense(num_class + 1, activation='softmax')(context)

    pred_model = Model(inputs_data, outputs)

    labels = Input(name='the_labels', shape=[46], dtype='float32')  # 46 is the max size of text length
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

    model = Model(inputs=[inputs_data, labels, input_length, label_length], outputs=loss_out)

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')

    hist = model.fit(x=[x_train, y_train, x_train_length, y_train_length], y=np.zeros(len(y_train)),
                     batch_size=default_config["batch_size"], epochs=100,
                     validation_data=([x_val, y_val, x_val_length, y_val_length], [np.zeros(len(y_val))]),
                     verbose=1)

    return model, pred_model, hist

### Load the best hyperparameter values

refer **[here](https://github.com/bdu-birhanu/Bayesian-Optimization-for-DNN-hyperparameter-tuning-)** to know more how you could select hyperparamnters' value with Bayesian Optimization

In [17]:
%pip install wget
import wget
import tensorflow as tf

hyp_url = "https://github.com/bdu-birhanu/HHD-Ethiopic/raw/main/src/all_code/best_hyp_train_attention.txt"
hyp_path = "best_hyp_train_attention.txt"
# Download the model file
wget.download(hyp_url, hyp_path)
with open(hyp_path) as f:
    data = f.read()
best_config = eval(data)# toget dictionary from the string which is saved in the disk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Model training and saving

This model can be trained multiple times by adjusting the value of the "rounds" parameter below. For example, if we set it to 2 (as given below), the model will run for two rounds, resulting in two separate model trainings with different weight initialization. Once the training is completed (in this demo, we just stop the training at the 5th epoch!!), you can save the trained models  and use them for prediction. To learn how to utilize the model for prediction, please refer to the instructions provided [here](https://github.com/bdu-birhanu/HHD-Ethiopic/blob/main/Test_HPopt-Attn-CTC.ipynb).

In [ ]:
path = '/content/model/'
rounds=2
for i in range(rounds):
    #model_train = hist_ocr_model()
    best_model, best_pred_model, best_history = hist_ocr_model(best_config, n_components=5, verbose=1)
    #model = model_train[0]
    best_pred_model.save(path + f'model_belened_CTC_deephyper_{i}.hdf5')

Epoch 1/100
1614/1614 [==============================] - 156s 90ms/step - loss: 86.2180 - val_loss: 70.9531
Epoch 2/100
1614/1614 [==============================] - 137s 85ms/step - loss: 62.2865 - val_loss: 61.6982
Epoch 3/100
1614/1614 [==============================] - 138s 85ms/step - loss: 61.0643 - val_loss: 60.9203
Epoch 4/100
1614/1614 [==============================] - 138s 86ms/step - loss: 60.0359 - val_loss: 59.4527
Epoch 5/100
1614/1614 [==============================] - 139s 86ms/step - loss: 58.9088 - val_loss: 58.3207
Epoch 6/100
 468/1614 [=======>......................] - ETA: 1:32 - loss: 57.9267